In [1]:
import numpy as np
import scipy.signal
import hfda

In [ ]:
###Mean and variance for a specific interval
def Mean_taux(signal,taux,hprime,h=0):
    return (1/hprime)*np.sum((signal[h+taux:taux+hprime+h]))

def Variance_taux(signal,taux,hprime,h=0):
    return (1/hprime)*np.sum((signal[h+taux:taux+hprime+h]-Mean_taux(signal,taux,hprime,h))**2)

In [ ]:
###I1 and I2:

def I1(c,signal,fs,h,hprime,t0=0):
    if (t0>c or t0<0):
        return "error : t0 is outside the range you want to calculate"
    elif (c>len(signal) or c<0):
        return "error : c is outside the range of your signal"
    else: 
        I1c = 0
        for i in range(t0,int(c*fs),int(c*fs)): 
            a =(1/h)*np.abs(Mean_taux(signal,i,hprime,h)-Mean_taux(signal,i,hprime))
            I1c = I1c+a
    return I1c

def I2(c,signal,fs,h,hprime,t0=0):
    if (t0>c or t0<0):
        return "error : t0 is outside the range you want to calculate"
    elif (c>len(signal) or c<0):
        return "error : c is outside the range of your signal"
    else: 
        I2c = 0
        for i in range(t0,int(c*fs),int(c*fs)): 
            a =(1/h)*np.abs(Variance_taux(signal,i,hprime,h)-Variance_taux(signal,i,hprime))
            I2c = I2c+a
    return I2c

In [ ]:
def discrepancies_mean_curve(signal,fs,h,hprime,t0=0):
    c = range(t0,len(signal),5)
    I1_val = np.array([])
    I2_val = np.array([])
    for j in c:
        I1_val = np.append(I1_val,I1(signal,fs,h,hprime,t0))
        I2_val = np.append(I2_val,I2(signal,fs,h,hprime,t0))
    return I1_val,I2_val